strings in python are sequences of unicode code points

In [3]:
ord('h') # ord function to get unidode code points in python

104

the problem with this type of encoding is
1. vocab size is too short
2. unicode standard is dynamic and new characters keeps on getting added based on new scripts

So, the need is something more stable
therefore, encodings.

3 types offered by the uincode consortium - utf-8, utf-16, utf-32

out of these, utf-8 is the most preferred one. But it generates variable byte stream which could be a problem when considering the language models context window which is limited for computational reasons. This will effect the next word prediction task

So the aim is to stick to the utf-8 encoding but also want to attend larger vocab size during attention and tune it as a hyperparameter.

Compress the bytes generated by utf-8 encoding

Therefore, the solution is byte pair encoding

So the idea is, we'll take the sequence in terms of bytes and pair them iteratively based on the frequency count thereby minting new tokens and thus increasing the vocabulaory size in the process. This compresses the input sequence and thus the token thereby having a knowledge rich vocabulary

In [ ]:
# raw text
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
raw_bytes = text.encode("utf-8")
print(raw_bytes)